In [1]:
#import basic library for some basic function
import numpy as np
# import library to perform file operation
import os #use to access the files 
import tarfile # use to extract dataset from zip files
import sys
import zipfile

In [2]:
import my_utils.loadDataset as dl
import my_utils.loadModel as lm
import my_utils.trainModel as tm

In [3]:
import torch
import torchvision

In [4]:
dl.setFolderLocation(datasets='/home/pragnesh/Dataset/',selectedDataset='IntelIC/',train='train',test='test')
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

In [5]:
#device1 = torch.device("cpu")
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device1)
newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
                        freeze_feature=False,device_l=device1)

/home/pragnesh/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


VGG16 Loaded
Linear(in_features=4096, out_features=1000, bias=True)


In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 ModelName='vgg16',model=newmodel,device_l=device1,
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

In [6]:
vgg16_features = lm.vgg16_feature_list
print(vgg16_features)

[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']


In [7]:
prune_count= [3,3, 6,6, 13,13,13, 26,26,26, 26,26,26]
j=0
for i in range(len(vgg16_features)):
    if vgg16_features[i]=='M':
        continue
    else:
        vgg16_features[i] = vgg16_features[i]-prune_count[j]
        j+=1
vgg16Model = lm.create_vgg_from_feature_list(vgg16_features)    
print(vgg16Model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(61, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(61, 122, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(122, 122, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(122, 243, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(243, 243, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(243, 243, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
"""Select the amount of feature we want to prune in each Layer"""
epochs = 10
max_lr = 1e-3
grad_clip = .2 
weight_decay = 1e-5 
L1=1e-5
itteration = 6
for  ittr in range(itteration):
    for i in range(len(Module)):
        prune.ln_structured(Module[i], name="weight", amount=pr[i]/itteration, n=1, dim=0)
    numberOfZero = 0
    numberOfElements = 0
    totalNumberOfZero=0
    totalNumberOfElements=0
    for i, j in zip(range(len(prunelist)),prunelist):
        numberOfZero = torch.sum(Module[i].weight == 0)
        totalNumberOfZero += numberOfZero
        numberOfElements = Module[i].weight.nelement()
        totalNumberOfElements += numberOfElements
        frac = 100. * float(torch.sum(Module[i].weight == 0))/float(Module[i].weight.nelement())
        with open(outfile,'a') as f:
            f.write((f"{j} Sparsity in {Module[i]} is \t{frac}"))
        
    with open(outfile,'a') as f:
        f.write(("Global Sparsity: {:.2f}%".format(100*float(numberOfZero)/float(numberOfElements))))

    print(f"******************** {ittr}**********************")
    #fit_one_cycle(ModelName,epochs, max_lr, model,weight_decay=0, 
    #             L1=0,grad_clip=None, opt_func=torch.optim.SGD):
    historyPrun = fit_one_cycle(MODEL_NAME,epochs, max_lr, newModel,  
                              grad_clip=grad_clip, 
                              weight_decay=weight_decay, L1=L1,
                              opt_func=opt_func
                              )